# **Đồ án cuối kì**
### **Mã lớp:** CS114.N21

### **Giảng viên:** Phạm Nguyễn Trường An
### **Đề tài:** Phân loại đánh giá của khách hàng trên Shopee

### **Thành viên nhóm H2O:**
- Nguyễn Quốc Huy Hoàng - 20520051
- Ngô Quang Vinh - 19522523

Import các thư viện cần thiết

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from transformers.tokenization_utils_base import TruncationStrategy
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import json
import requests
from sklearn.model_selection import train_test_split
import re
import underthesea # Thư viện tách từ
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from joblib import dump
from pprint import pprint

## Xử lý dữ liệu

### Đọc dữ liệu sau đó chia làm 2 tập train/test

In [ ]:
# Đọc dữ liệu từ file Excel
file_path = "Book1.xlsx"
data_frame = pd.read_excel(file_path, header=None)

# Truy cập vào cột thứ nhất và thứ hai
labels = data_frame.iloc[:, 0].tolist()
reviews = data_frame.iloc[:, 1].tolist()

X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=20520051, shuffle=True)
pprint("X_train[0]")
pprint(X_train[0])

'X_train[0]'
'Kêu 1 cặp 2 cái đặt x5 mà giao đc 5 cái shop làm ăn kiểu gì v'


In [ ]:
tichcuc = "sản phẩm tốt chất lượng tốt "
tieucuc = "sản phẩm tệ kém chất lượng "
with open('icon_tieu_cuc.txt', 'r', encoding='utf-8') as file:
    icon_tieu_cuc = file.read()
print(icon_tieu_cuc)
with open('icon_tich_cuc.txt', 'r', encoding='utf-8') as file:
    icon_tich_cuc = file.read()
print(icon_tich_cuc)

🙂👿😠😡🙁😒😫😕😩😣😟😭😢😖😔😞☹️🤧🤒😷🤕😵🤢🤠🤡👹👺👻💀👽😰😨😧🤥😈🙃👎🤬👋😌😏😬💩😓😤😮‍💨😑
😀😊😉😍😘😗😙🤗😚😛😝😜😋🤑😎😇♥️❤️💛💚💙💜🖤💖💝🤩🥰😅💞💋🤗😋🥳



### Chuẩn hóa dữ liệu

In [ ]:
# Hàm chuẩn hoá câu
def standardize_data(row):
    # Icon biểu tượng
    for val in row:
        if val in icon_tich_cuc:
            row = tichcuc + row
        if val in icon_tieu_cuc:
            row = tieucuc + row
    # Xoá hết những cái không phải chữ và số
    row =  re.sub(r"[^\w\s]", " ", row)

    # # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    # row = re.sub(r"[\.,\?]+$-", "", row)
    # # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    # row = row.replace(",", " ").replace(".", " ") \
    #     .replace(";", " ").replace("“", " ") \
    #     .replace(":", " ").replace("”", " ") \
    #     .replace('"', " ").replace("'", " ") \
    #     .replace("!", " ").replace("?", " ") \
    #     .replace("-", " ").replace("?", " ")
    # # Nếu có nhiều khoảng trắng thì rút gọn còn 1
    row = row.replace('\n'," ")
    row = re.sub(r"\s+", " ", row)
    # Đưa hết về chữ thường
    row = row.strip().lower()
    str = ""
    str += row[0]
    for i in range(1, len(row)):
        if row[i] != row[i - 1]:
            str += row[i]
    # pprint(str)
    # print(ascii(row[16]))
    # Chuyển thành dạng không dấu
    # str = unidecode(str)
    return str

### Lấy danh sách stop word của tiếng Việt

In [ ]:
# Hàm load danh sách các từ vô nghĩa: lắm, ạ, à, bị, vì..
sw = []
with open("vietnamese-stopwords - Copy.txt", encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    sw.append(line.replace("\n",""))

### Chuyển văn bảng thành dạng vetor đặc trưng

In [ ]:
# model_trans = AutoModel.from_pretrained("Fsoft-AIC/videberta-xsmall")
# tokenizer = AutoTokenizer.from_pretrained("Fsoft-AIC/videberta-xsmall")

# model_trans = AutoModel.from_pretrained("Fsoft-AIC/videberta-base")
# tokenizer = AutoTokenizer.from_pretrained("Fsoft-AIC/videberta-base")

model_trans = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# model_trans = AutoModel.from_pretrained("vinai/phobert-large")
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")

# model_trans = AutoModel.from_pretrained("vinai/phobert-base-v2")
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Hàm tạo ra bert features
def make_bert_features(v_text):
    v_tokenized = []
    max_len = 150 # Mỗi câu dài tối đa 200 từ
    for i_text in v_text:
        # print("Đang xử lý line = ", i_text)
        # Chuẩn hóa

        i_text = i_text.replace("\n"," ")
        i_text = standardize_data(i_text)

        # # Phân thành từng từ
        # line = underthesea.word_tokenize(i_text)

        # # Lọc các từ vô nghĩa
        # # filtered_sentence = [w for w in line if not w in sw]
        # filtered_sentence = line

        # # Ghép lại thành câu như cũ sau khi lọc
        # line = " ".join(filtered_sentence[:100])
        # line = underthesea.word_tokenize(line, format="text")
        line = ViTokenizer.tokenize(i_text)[:150]


        # print("Word segment  = ", line)
        # Tokenize bởi BERT
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    padded = []
    # Chèn thêm số 1 vào cuối câu nếu như không đủ từ hoặc xóa nếu dư
    for line in v_tokenized:
        if len(line) < max_len:
            padded.append(line + [1] * (max_len - len(line)))
        else:
            padded.append(line[: max_len])
    padded = np.array(padded)

    # print('padded:', padded[0])
    # print('len padded:', padded.shape)

    # Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
    attention_mask = np.where(padded == 1, 0, 1)
    # print('attention mask:', attention_mask[0])

    # Chuyển thành tensor
    padded = torch.tensor(padded).to(torch.long)
    print("Padd = ",padded.size())
    attention_mask = torch.tensor(attention_mask)

    # Lấy features dầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = model_trans(input_ids= padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    print(v_features.shape)
    return v_features

In [ ]:
# Xử lý dữ liệu tập train
X_train = make_bert_features(X_train)
X_train = np.array(X_train)
y_train = np.array(y_train)

Padd =  torch.Size([1948, 150])


In [ ]:
# Xử lý dữ liệu tập test
X_test = make_bert_features(X_test)
X_test = np.array(X_test)
y_test = np.array(y_test)

Padd =  torch.Size([487, 150])
(487, 384)


## Mô hình phân lớp


### SVM

#### Xây dựng và huấn luyện mô hình

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

#### Dự đoán và đánh giá

In [ ]:
# Dự đoán nhãn cho tập kiểm tra
y_pred = svm_model.predict(X_test)

# Đánh giá mô hình
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

F1 score: 0.7334200260078023


#### Tinh chỉnh mô hình

In [ ]:
# Tune model bằng grid search
parameters = {
                'kernel': ('linear', 'rbf'),
                'C': [0.5, 1, 2, 4],
                'gamma': ['scale']#, 0.125, 0.25, 0.5, 1, 2, 4]
            }

clf = GridSearchCV(SVC(), param_grid=parameters,  cv = 5, scoring='f1', n_jobs=-1)
grid_search = clf.fit(X_train, y_train)

# best prarams
print('best prarams:', clf.best_params_)

svm_best_model = grid_search.best_estimator_
y_pred = svm_best_model.predict(X_test)

# Đánh giá mô hình
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

best prarams: {'C': 0.5, 'gamma': 'scale', 'kernel': 'linear'}
F1 score: 0.7334200260078023


In [ ]:
# Save model
# dump(svm_model, 'save_model.joblib')

['save_model.joblib']

### Logistic Regression

#### Xây dựng và huấn luyện vô hình

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

#### Dự đoán và đánh giá

In [ ]:
# Dự đoán nhãn cho tập kiểm tra
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

F1 score: 0.7334200260078023


#### Tinh chỉnh mô hình

In [ ]:
model = LogisticRegression()

# Thiết lập lưới tham số cần tìm kiếm
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['lbfgs', 'liblinear', 'saga']
}

# Tìm kiếm lưới các tham số tốt nhất
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

# In ra bộ tham số tốt nhất và đánh giá mô hình
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Đánh giá mô hình trên dữ liệu kiểm tra
y_pred = grid_search.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

Best Parameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score: 0.7213650115962603
F1 score: 0.7334200260078023


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\

### Radom Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

# Định nghĩa các tham số cần tinh chỉnh và giá trị để thử nghiệm
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}


# Tạo đối tượng Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv = 5, scoring='f1', n_jobs=-1)

# Tiến hành Grid Search trên dữ liệu huấn luyện
grid_search.fit(X_train, y_train)

# In ra các thông số tốt nhất từ Grid Search
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Dự đoán nhãn cho dữ liệu kiểm tra sử dụng mô hình tốt nhất
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Đánh giá độ chính xác
accuracy = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best parameters:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best score:  0.7213650115962603
Accuracy: 0.7334200260078023


### XGB

In [ ]:
from xgboost import XGBClassifier
# Khởi tạo mô hình XGBClassifier
model = XGBClassifier()

# Huấn luyện mô hình trên tập huấn luyện
model.fit(X_train, y_train)

# Dự đoán nhãn cho tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá độ chính xác của mô hình
accuracy = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8939929328621908


In [ ]:
# model = XGBClassifier()

# # Định nghĩa siêu tham số và giá trị để tinh chỉnh
# param_grid = {
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.1, 0.01, 0.001],
#     'n_estimators': [100, 500, 1000]
# }

# # Sử dụng Grid Search để tìm siêu tham số tốt nhất
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv=3, n_jobs=4)
# grid_search.fit(X_train, y_train)

# # Lấy siêu tham số tốt nhất và tạo mô hình với siêu tham số đó
# best_params = grid_search.best_params_
# model = XGBClassifier(**best_params)

# # Huấn luyện mô hình trên toàn bộ tập huấn luyện
# model.fit(X_train, y_train)

# # Dự đoán nhãn cho tập kiểm tra
# y_pred = model.predict(X_test)

# # Đánh giá độ chính xác của mô hình
# accuracy = f1_score(y_test, y_pred)
# print("Accuracy:", accuracy)